In [ ]:
import torch
from torch import nn
import torchvision
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from torch.utils.data import Dataset, DataLoader
import cv2
import torchtext
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score

In [ ]:
drive.mount("/content/drive")
data_path = "/content/drive/My Drive/Colab Notebooks/data/"
train_path = "/content/drive/My Drive/Colab Notebooks/data/train.jsonl"
val_path = "/content/drive/My Drive/Colab Notebooks/data/dev.jsonl"
test_path = "/content/drive/My Drive/Colab Notebooks/data/test.jsonl"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class ImageDataset(Dataset):
  def __init__(self, file_path, data_path):
    self.file = pd.read_json(file_path, lines=True)
    self.data_path = data_path
  
  def __len__(self):
    return len(self.file)
  
  def __getitem__(self,idx):
    img = Image.open(self.data_path + self.file["img"][idx]).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = preprocess(img)
    # img = img.permute(1,2,0)
    label = self.file["label"][idx]
    label = torch.tensor(label, dtype=float)
    sample = {}
    sample = {"img":img,"label":label}
    return sample

In [ ]:
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
# model = nn.Sequential(
#     model, 
#     nn.Linear(1000,1),
#     nn.Softmax()
#   )
# model.to(device)

model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet152', pretrained=True)
for p in model.parameters():
    p.requires_grad = False
model = nn.Sequential(
          model, 
          nn.Linear(1000,1000), 
          nn.Linear(1000,100),
          nn.Linear(100,1),
          nn.Sigmoid()
          )
model.cuda()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

In [ ]:
train_imgs = ImageDataset(train_path, data_path)
val_imgs = ImageDataset(val_path, data_path)

train_loader = DataLoader(train_imgs, batch_size=64, shuffle=True)
val_loader = DataLoader(val_imgs, batch_size=64, shuffle=True)

In [ ]:
def binary_accuracy(y_real, y_pred):
  y_pred_label = torch.round(y_pred)
  correct_results_sum = (y_pred_label == y_real).sum().float()
  acc = correct_results_sum/y_real.shape[0]
  return acc

In [ ]:
def validate (model, val_loader,  early_stop, best, prev_acc):
  model.eval()
  with torch.no_grad():
    val_acc = 0.0
    n = 0
    for j,b in enumerate(val_loader):
      val_batch = b['img']
      val_labels = b['label']
      val_batch = val_batch.cuda().float()
      val_labels = val_labels.cuda().float()
      val_out = model(val_batch)
      val_acc += binary_accuracy(val_labels, val_out.squeeze(1))
      n +=1
    val_acc = val_acc/n
    if val_acc > prev_acc:
      torch.save(model.state_dict(), '/content/drive/My Drive/best_model_cnn.pt')
    if val_acc < best.min():
      early_stop = True
    else:
      best[0]=val_acc
      best.sort()
  print('\t\tValidation accuracy: %f'%val_acc)
  return val_acc

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 40
 
acc_hiss = []
loss_hiss = []
running_acc = 0.0
running_loss = 0.0
early_stop = False
best = np.zeros(10)
prev_acc = 0.0
for epoch in range(num_epochs):
  running_acc = 0.0
  running_loss = 0.0
  print("---------------------------------------------------------------------")
  print("Epoch: ", epoch)
  for i, batches in enumerate(train_loader):
    batch = batches['img']
    labels = batches['label']
 
    optimizer.zero_grad()
    batch = batch.cuda().float()
    labels = labels.cuda().float()
 
    outputs = model(batch)
    loss = criterion(outputs, labels.unsqueeze(1))
    loss.backward()
    optimizer.step()
 
    accuracy = binary_accuracy(outputs, labels.unsqueeze(1))
    running_acc += accuracy.item()
    running_loss += loss.item()
    acc_hiss.append(accuracy.item())
    loss_hiss.append(loss.item())
    if i%20==0 and i >0:
      print("\tLoss = %f , Acc=%f"%( running_loss/20, running_acc/20))
      running_acc = 0.0
      running_loss = 0.0
  if epoch % 3 == 0 and i>2:
      prev_acc = validate(model, val_loader, early_stop, best, prev_acc)
      model.train()
  print("---------------------------------------------------------------------")
  if early_stop == True:
    break;          
print("Finish training.")

---------------------------------------------------------------------
Epoch:  0
	Loss = 0.885118 , Acc=0.582031
	Loss = 0.692274 , Acc=0.605469
	Loss = 0.661617 , Acc=0.617188
	Loss = 0.679546 , Acc=0.588281
	Loss = 0.663261 , Acc=0.625000
	Loss = 0.644552 , Acc=0.659375
		Validation accuracy: 0.501953
---------------------------------------------------------------------
---------------------------------------------------------------------
Epoch:  1
	Loss = 0.690220 , Acc=0.660937
	Loss = 0.645685 , Acc=0.633594
	Loss = 0.658675 , Acc=0.603125
	Loss = 0.645261 , Acc=0.633594
	Loss = 0.623388 , Acc=0.663281
	Loss = 0.645249 , Acc=0.638281
---------------------------------------------------------------------
---------------------------------------------------------------------
Epoch:  2
	Loss = 0.668650 , Acc=0.681250
	Loss = 0.623247 , Acc=0.664844
	Loss = 0.621924 , Acc=0.664844
	Loss = 0.657031 , Acc=0.625781
	Loss = 0.638005 , Acc=0.634375
	Loss = 0.645822 , Acc=0.626563
------------

KeyboardInterrupt: ignored

In [ ]:
# dataiter = iter(val_loader)
# d = dataiter.next()
# imgs = d['img']
# labels = d['label']
# model.cpu()
# model.eval()
# with torch.no_grad():
#   out = model(imgs)

In [ ]:
nepochs = np.arange(len(acc_hiss))
plt.figure(figsize=(15,15))
plt.title('Accuracy')
plt.plot(nepochs, acc_hiss)
plt.figure(figsize=(15,15))
plt.title('Loss')
plt.plot(nepochs, loss_hiss)
